In [ ]:
import os
import scipy.io
import pandas as pd

In [ ]:
# Definir o caminho do diretório principal onde estão os arquivos .mat
root_directory = r'C:\Clone\CWRU-dataset'

# Inicializar uma lista para armazenar os dados
data = []

# Percorrer todas as pastas e subpastas
for root, dirs, files in os.walk(root_directory):
    relative_path = os.path.relpath(root, root_directory)
    first_folder = relative_path.split(os.sep)[0]
    
    for file in files:
        if file.endswith('.mat'):

            fault_type = 'normal'
            fault_location = 'normal'
            fault_size = 0
            sample_freq = True


            # Carregar o arquivo .mat
            file_path = os.path.join(root, file)
            mat_data = scipy.io.loadmat(file_path)

            if first_folder == "Normal":
                fault_type = 'normal'
                fault_location = 'normal'
                fault_size = 0
                sample_freq = False

            elif first_folder == "12k_Drive_End_Bearing_Fault_Data":
                relative_path = os.path.relpath(root, root_directory)
                fault_location = 'DE'
                sample_freq = True

            elif first_folder == "48k_Drive_End_Bearing_Fault_Data":
                fault_location = 'DE'
                sample_freq = False

            elif first_folder == "12k_Fan_End_Bearing_Fault_Data":
                fault_location = 'FE'
                sample_freq = True
                

            
            

            if first_folder != 'Normal':
                relative = os.path.relpath(file, relative_path)

                fault_size = int(relative_path.split(os.sep)[2])
                fault_type = relative_path.split(os.sep)[1]
                
                




            load = file[-5]

            
            # Inicializar dicionário para armazenar os sinais DE e FE
            signals = {'fault_type': fault_type, 'fault_location' : fault_location, 'fault_size' : fault_size, 'load':load, '12K': sample_freq, 'DE': None, 'FE': None, 'BA': None}
            
            # Percorrer as chaves e valores do .mat para identificar as colunas desejadas
            for key in mat_data:
                if 'FE_time' in key:
                    signals['FE'] = mat_data[key].flatten().tolist()
                elif 'DE_time' in key:
                    signals['DE'] = mat_data[key].flatten().tolist()
                elif 'BA_time' in key:
                    signals['BA'] = mat_data[key].flatten().tolist()
            
            # Adicionar o sinal ao nosso dataset se tiver pelo menos um dos sinais
            data.append(signals)

# Criar o DataFrame com as listas dos sinais
df = pd.DataFrame(data)

# Exibir o DataFrame
df.head()


In [ ]:
df.replace(
    to_replace={
        'B': 'ball',
        'OR': 'outer',
        'IR': 'inner'},
    inplace=True)
df['fault_type'].unique()

In [ ]:
# Defining labels
labels = pd.Series([df['fault_type'][i] + '-' + str(df['fault_size'][i])
                    for i in range(len(df))]).replace({'normal-0': 'normal'})
classes = sorted(labels.unique())
classes

In [ ]:
# Adding class labels in the resulting df
df.insert(4, 'label', [classes.index(label) for label in labels])


In [ ]:
print(df.shape)
df.head(10)

In [ ]:
df.to_pickle(r"G:\Meu Drive\Scripts\Pos\Aprendizadomaquina\Trabalho_final\data\cwru.pkl")